In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, GridSearchCV, cross_val_predict
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from skopt import BayesSearchCV, gp_minimize, forest_minimize, gbrt_minimize, dummy_minimize
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from skopt.utils import use_named_args
from skopt import Optimizer
import numpy as np
import pyswarm
from pyswarm import pso


In [5]:
# Load the dataset
data = pd.read_csv('clean_data (copy).csv')
X = data.drop('target', axis=1)  # Drop the target column
y = data['target']

In [6]:
# Define parameters for grid search
param_grid = {
    'hidden_layer_sizes': [(16,), (32,), (64,)],
    'activation': ['relu', 'tanh', 'logistic'],
    'solver': ['adam', 'sgd'],
    'learning_rate_init': [0.001, 0.01, 0.1],
    'max_iter': [10000],  # Adjusted max_iter
    'alpha': [0.0001, 0.001, 0.01]  # Regularization parameter
}

In [7]:
# K-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True)


In [8]:
# Dictionary to store predictions
predictions = {}

In [9]:
# Define MLPRegressor model directly
mlp_model = MLPRegressor()  # Updated model definition

In [10]:
# Grid Search
grid_search = GridSearchCV(estimator=mlp_model, param_grid=param_grid, cv=kfold, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X, y)
best_model = grid_search.best_estimator_
predictions['MLP_GridSearch'] = cross_val_predict(best_model, X, y, cv=kfold)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


In [13]:
# Define parameter search spaces
param_space = {
    'hidden_layer_sizes': Integer(16, 64),
    'activation': ['relu', 'tanh', 'logistic'],
    'solver': ['adam', 'sgd'],
    'learning_rate_init': Real(0.001, 0.1, prior='log-uniform'),
    'max_iter': [10000],
    'alpha': Real(0.0001, 0.01, prior='log-uniform')
}

In [14]:
# Random Search
random_search = BayesSearchCV(
    estimator=mlp_model,
    search_spaces=param_space,
    n_iter=20,
    cv=kfold,
    scoring='neg_mean_squared_error',
    random_state=42,
    verbose=1
)
random_search.fit(X, y)
predictions['RandomSearch'] = cross_val_predict(random_search.best_estimator_, X, y, cv=kfold)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

In [17]:
# Hyperband
def hyperband_objective(space):
    mlp = MLPRegressor(hidden_layer_sizes=space['hidden_layer_sizes'],
                      activation=space['activation'],
                      solver=space['solver'],
                      learning_rate_init=space['learning_rate_init'],
                      max_iter=space['max_iter'],
                      alpha=space['alpha'])

    mse = cross_val_predict(mlp, X, y, cv=kfold)
    return {'loss': -np.mean(mse), 'status': STATUS_OK}

hyperband_space = {
    'hidden_layer_sizes': hp.randint('hidden_layer_sizes', 16, 64),
    'activation': hp.choice('activation', ['relu', 'tanh', 'logistic']),
    'solver': hp.choice('solver', ['adam', 'sgd']),
    'learning_rate_init': hp.loguniform('learning_rate_init', np.log(0.001), np.log(0.1)),
    'max_iter': 10000,
    'alpha': hp.loguniform('alpha', np.log(0.0001), np.log(0.01))
}

hyperopt_trials = Trials()
hyperopt_best = fmin(fn=hyperband_objective,
                      space=hyperband_space,
                      algo=tpe.suggest,
                      max_evals=20,
                      trials=hyperopt_trials)
best_hyperband_model = MLPRegressor(hidden_layer_sizes=hyperopt_best['hidden_layer_sizes'],
                                    activation=['relu', 'tanh', 'logistic'][hyperopt_best['activation']],
                                    solver=['adam', 'sgd'][hyperopt_best['solver']],
                                    learning_rate_init=hyperopt_best['learning_rate_init'],
                                    max_iter=10000,
                                    alpha=hyperopt_best['alpha'])
predictions['Hyperband'] = cross_val_predict(best_hyperband_model, X, y, cv=kfold)

100%|██████████| 20/20 [00:32<00:00,  1.61s/trial, best loss: -9592648522.240143]


In [19]:
# Define parameter search spaces
param_space = [
    Integer(16, 64, name='hidden_layer_sizes'),
    Real(0.001, 0.1, prior='log-uniform', name='learning_rate_init'),
    Real(0.0001, 0.01, prior='log-uniform', name='alpha')
]


In [22]:
# BO-TPE
@use_named_args(param_space)
def bo_tpe_objective(hidden_layer_sizes, learning_rate_init, alpha):
    mlp = MLPRegressor(hidden_layer_sizes=int(hidden_layer_sizes),
                       learning_rate_init=learning_rate_init,
                       alpha=alpha,
                       max_iter=10000)

    mse = cross_val_predict(mlp, X, y, cv=kfold)
    return -np.mean(mse)  # Minimize negative mean squared error

bo_tpe = forest_minimize(bo_tpe_objective, param_space, n_calls=20, random_state=42)

best_bo_tpe_model = MLPRegressor(hidden_layer_sizes=int(bo_tpe.x[0]),
                                 learning_rate_init=bo_tpe.x[1],
                                 alpha=bo_tpe.x[2],
                                 max_iter=10000)

predictions['BO_TPE'] = cross_val_predict(best_bo_tpe_model, X, y, cv=kfold)

In [25]:
# BO-GP
@use_named_args(param_space)
def bo_gp_objective(hidden_layer_sizes, learning_rate_init, alpha):
    mlp = MLPRegressor(hidden_layer_sizes=int(hidden_layer_sizes),
                       learning_rate_init=learning_rate_init,
                       alpha=alpha,
                       max_iter=10000)

    mse = cross_val_predict(mlp, X, y, cv=kfold)
    return np.mean(mse)  # Minimize mean squared error

bo_gp = gp_minimize(bo_gp_objective, param_space, n_calls=20, random_state=42)


In [31]:
# Define the objective function for PSO
def pso_objective(x):
    mlp = MLPRegressor(hidden_layer_sizes=int(x[0]),
                       learning_rate_init=x[1],
                       alpha=x[2],
                       max_iter=10000)

    mse = cross_val_predict(mlp, X, y, cv=kfold)
    return np.mean(mse)

# Define the bounds for PSO
lower_bound = [16, 0.001, 0.0001]
upper_bound = [64, 0.1, 0.01]

# Perform PSO optimization
pso_result = pso(pso_objective, lower_bound, upper_bound, swarmsize=20, maxiter=20)

# Extract the optimal hyperparameters
optimal_hidden_layer_sizes = int(pso_result[0][0])
optimal_learning_rate_init = pso_result[0][1]
optimal_alpha = pso_result[0][2]


# Create MLPRegressor with optimal hyperparameters
best_pso_model = MLPRegressor(hidden_layer_sizes=optimal_hidden_layer_sizes,
                              learning_rate_init=optimal_learning_rate_init,
                              alpha=optimal_alpha,
                              max_iter=10000)

# Perform cross-validation predictions
predictions['PSO'] = cross_val_predict(best_pso_model, X, y, cv=kfold)

Stopping search: maximum iterations reached --> 20


In [32]:
# Save predictions to CSV files
for technique, pred_values in predictions.items():
    df = pd.DataFrame({'Actual': y, 'Predicted': pred_values})
    df.to_csv(technique + '_predictions.csv', index=False)